In [1]:
import numpy as np

In [2]:
class RegressionTree():

    def __init__(self, max_depth = 5, min_samples_split = 2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    class Node:
        def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right
            self.value = value 

    def __best_split(self, X, y):
        best_feature = None
        best_threshold = None
        best_mse = np.inf

        _, n_features = X.shape

        for feature in range(n_features):
            thresholds = np.unique(X[:,feature])

            for threshold in thresholds:
                _, y_left, _, y_right = split_dataset(X,y,feature,threshold)

                if len(y_left) == 0 or len(y_right) == 0: continue

                mse_left = MSE(y_left)
                mse_right = MSE(y_right)

                weighted_mse = (len(y_left)/len(y) * mse_left + len(y_right)/len(y) * mse_right)

                if weighted_mse < best_mse:
                    best_mse = weighted_mse
                    best_feature = feature
                    best_threshold = threshold
        
        return best_feature, best_threshold, best_mse
    
    def __build_tree(self, X, y, depth=0):
        if depth >= self.max_depth or len(y) < self.min_samples_split:
            return self.Node(value=np.mean(y))
        
        feature, threshold, mse = self.__best_split(X,y)

        if feature is None or mse >= MSE(y):
            return self.Node(value=np.mean(y))
        
        X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)

        left_child = self.__build_tree(X_left, y_left, depth+1)
        right_child = self.__build_tree(X_right, y_right, depth+1)

        return self.Node(feature, threshold, left_child, right_child)
    
    def __predict_sample(self, x, node: Node):
        if node.left is None and node.right is None:
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self.__predict_sample(x, node.left)
        else:
            return self.__predict_sample(x, node.right)
        
    def fit(self, X, y):
        self.root = self.__build_tree(X, y)

    def predict(self, X):
        return np.array([self.__predict_sample(x, self.root) for x in X])
